# Pandas (and matplotlib) in Action

Import our libraries:

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('max_rows', 25)
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (10, 4)

(Remember, you can just copy the code above and past it into a new notebook in order to get started)

## First real example: the weather! ##

Before we try using pandas with word counts, let's look at some more typical data. Here is a dataset of several years of hourly weather and pollution readings in Beijing: the [Beijing PM2.5 data set](https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data). It's from the UCI Machine Learning Repository, which I mentioned last class.

To start, download the CSV file using **[this link](https://archive.ics.uci.edu/ml/machine-learning-databases/00381/PRSA_data_2010.1.1-2014.12.31.csv)** and **save it in the same folder as your Jupyter Notebook.** 

If you're curious the people who produced the data also wrote a paper on it:

> [Liang, X., Zou, T., Guo, B., Li, S., Zhang, H., Zhang, S., Huang, H. and Chen, S. X. (2015). Assessing Beijing's PM2.5 pollution: severity, weather impact, APEC and winter heating. Proceedings of the Royal Society A, 471, 20150257.](http://www.stat-center.pku.edu.cn/Stat/Uploads/Files/[20160114_1120]Beijing%20Air-Quality%20Assessment%20Report.pdf).
    
The paper has some technical content, but overall it's very readable and skimming it will help you understand the data a bit better. (Data biography alert!) 

The larger point, though, is that pandas makes it *very* easy to use data in CSV format. Just use the `read_csv()` function and pass it the filename of your data:

In [ ]:
df = pd.read_csv("./PRSA_data_2010.1.1-2014.12.31.csv")

Pandas does a good job of guessing the correct data types for the values in the CSV file. (If Pandas gets it wrong, though, don't lose hope: [here's a good overview of strategies you can use to clean it up](https://github.com/KarrieK/pandas_data_cleaning).)

Let's take a look at the DataFrame we ended up with:

In [ ]:
df

You can see that because there are so many rows in this DataFrame (43,824!), Pandas shows only a subset. But it's enough for us to get an idea of what the DataFrame looks like.

The `.info()` method shows us the rows and their data types:

In [ ]:
df.info()

The `int64`, `float64`, etc. data types are specific to Pandas, and are not the same thing as their regular Python equivalent. (Actually, they're specific to [Numpy](http://www.numpy.org/), but that's a different story.)

Of course, Pandas can't tell us what the data in these columns *mean*. For that, we need to consult the documentation that accompanies the data. Copying and pasting from the web page linked to above, here are the meanings for each field:

* No: row number
* year: year of data in this row
* month: month of data in this row
* day: day of data in this row
* hour: hour of data in this row
* pm2.5: PM2.5 concentration (ug/m^3)
* DEWP: Dew Point (deg C)
* TEMP: Temperature (deg C)
* PRES: Pressure (hPa)
* cbwd: Combined wind direction
* Iws: Cumulated wind speed (m/s)
* Is: Cumulated hours of snow
* Ir: Cumulated hours of rain

> Note that these aren't *universal* names for these fields. You can't expect to download a different data set from another set of researchers that records similar phenomena and expect that file to use (e.g.) `TEMP` as the column name for temperature.

As with Series in general, we can grab one of these columns and use `.describe()` to get a general overview of what data it contains:

In [ ]:
df["pm2.5"].describe()

This tells us, e.g., that the "average" level of PM2.5 concentration in Beijing over the four-year period of the data was 98.6, with half of days being over 72 and half under. The highest PM2.5 recorded in the data was 994.

Looking at the plot for the `pm2.5` column, you can kind of make out yearly cycles in PM2.5 concentration:

In [ ]:
df.plot(y="pm2.5")

We can do the same analysis with the other fields. For example, here's a plot of temperature readings for each hour:

In [ ]:
df.plot(y="TEMP")

Plotting these two together shows an interesting pattern, maybe:

In [ ]:
df.plot(y=["pm2.5", "TEMP"])

It looks like when temperature dips, pm2.5 spikes! (There are various statistical ways to confirm this suspicion, but for now we're going to stick with drawing the graphs.)

### Histograms

I like histograms for a quick overview of how your data are *distributed*. Understanding distribution helps you better reason about how often particular values are found in your data, and helps you easily formulate hypotheses about the phenomena your data is tracking. 

Let's look at a histogram of temperature data in our Beijing data set, using the `hist` plot kind:

In [ ]:
df.plot(kind="hist", y="TEMP")

Each bar in this graph corresponds to a "bin" of values surrounding the value on the X axis. When drawing a histogram, Pandas looks at each item in the data and puts it in the bin corresponding to the closest value. So for example, the graph above tells us that there are a lot of temperature readings (~8000) around 20 degrees C, but very few (less than 300) readings around 40 degrees C. You can increase the "resolution" of the histogram by providing a `bins` named parameter:

In [ ]:
df.plot(kind="hist", y="TEMP", bins=20)

From this graph, we might hypothesize that a way to characterize Beijing temperatures is that they mostly cluster in either the 20—30 degrees C range, or the -5 to +5 degrees C range. Temperatures above 40 degrees C or below -20 degrees C are rare. The histogram for temperatures looks very different from the histogram for PM2.5:

In [ ]:
df.plot(kind="hist", y="pm2.5", bins=20)

This histogram shows that while there are a number of outliers, by far most of the PM2.5 readings are in the 0–200 range.

### Scatter plots

A scatter plot is an easy way to confirm your suspicion that two columns in your data set are somehow related. In a scatter plot, you select two columns, and every row in the data set becomes a point in a two-dimensional space, based on the value of those two columns in the row. You need to specify both columns using the `x` and `y` named parameters. So, for example, here's a scatter plot with temperature and dew point:

In [ ]:
df.plot(kind="scatter", x="DEWP", y="TEMP")

Each dot in this scatterplot represents a row from the DataFrame. (Sometimes these dots are so dense that they appear to form solid masses or lines.) This scatter plot shows that as the temperature rises, so does the dew point ([as you might expect from the definition of dew point](https://en.wikipedia.org/wiki/Dew_point)). One way to talk about this relationship is to say that the values in these two columns are *correlated*.

However, drawing a scatter plot of PM2.5 concentration with the cumulative wind speed shows an inverse relationship:

In [ ]:
df.plot(kind="scatter", x="pm2.5", y="Iws")

You might interpret this graph as an indication that in general, as the wind speed goes up, the PM2.5 concentration falls. (This is intuitively true, and the authors of the paper go into a bit more detail about this effect in particular.) A scatter plot of PM2.5 and dew point also shows a correlation:

In [ ]:
df.plot(kind="scatter", x="DEWP", y="pm2.5")

Again, this is a Pandas tutorial, not a statistics tutorial, so take these characterizations with a grain of salt. My goal here is to show you how histograms and scatter plots are good starting points for getting a "feel" for your data and how the variables might be related.

Note that this approach is also helpful for things like word counts or topics (over time, in different subsets of a corpus, in two separate corpora, etc) as much as it does for numbers.

### Answering questions with selection

Let's say we wanted to find out how many readings in the data had a PM2.5 concentration of greater than 500. One easy way to do this is to use Boolean indexing, as discussed last class. The following expression gives us a Boolean Series, with True values for every row with a PM2.5 greater than 400:

In [ ]:
df["pm2.5"] > 400

And then we can use that to subscript the DataFrame, giving us a new DataFrame with only the rows that meet the condition:

In [ ]:
df[df["pm2.5"] > 400]

Pandas tells us that there are 545 such rows. With this limited DataFrame, we can still draw plots! So, for example, if we wanted to see a temperature histogram just for these days:

In [ ]:
df[df["pm2.5"] > 400].plot(kind="hist", y="TEMP", bins=20)

Comparing this distribution to the rows where PM2.5 is less than 400:

In [ ]:
df[df["pm2.5"] < 400].plot(kind="hist", y="TEMP", bins=20)

You can see that the two distributions are quite different, with the temperatures on days with high PM2.5 concentrations being lower on average.

### Value counts and bar charts

The `cbwd` indicates the prevailing wind direction, which the researchers have narrowed down to four distinct values: NE (northeast), NW (northwest), SE (southeast) and "cv" ("calm or variable"). They outline the reasons for recording the data this way in their paper. The values in this column, unlike the values in the other columns, consist of a discrete set, rather than continuous numbers. As such, Pandas will be confused if we ask for a plot:

In [ ]:
df["cbwd"].plot()

The "no numeric data to plot" error is Pandas saying, "hey you wanted me to draw a graph, but there are no numbers in this field, what gives." Probably the best way to visualize discrete values is by *counting them* and then drawing a bar graph. As discussed earlier, the `.value_counts()` method returns a Series that counts how many times each value occurs in a column:

In [ ]:
df["cbwd"].value_counts()

Plotting this data as a bar chart shows us how many times each of these discrete values were recorded:

In [ ]:
df["cbwd"].value_counts().plot(kind="barh")

This should already be getting you thinking about word counts, which is where we'll end up today. But before that, a few more topics to cover:

### Sorting

Sorting in pandas is quite easy. This is how you sort by a particular value:

In [ ]:
sorted_df = df.sort_values(by=["pm2.5"], ascending=False)

sorted_df

### Group by

You can also group the data by a particular column in this way:

In [ ]:
monthly_mean_df = df.groupby("month").mean()

In [ ]:
monthly_mean_df.plot(kind="bar", y=["pm2.5", "Iws"])

OK. Now we've learned most of the basic pandas / matplotlib functionality. Let's see what we can do with some word counts! 

## Second real example: Colored Conventions (for the last time). 

Unless you decide to use the Colored Conventions corpus for your final project, this is the last time we'll be using this dataset. So once again, let's create our list of docs for analysis. 

In [ ]:
# create our list of docs

import os

base_dir = "./2019-09-ccp-corpus-0.3/ccprecords/" # NOTE: Your path may be different!!!

all_docs = [] # our list which will store the text of each doc; empty for now

docs = os.listdir(base_dir) # get a list of all the files in the directory

docs.sort() # imprecise but good enough for a tutorial

for doc in docs: # iterate through the docs
    if not doc.startswith('.'): # get only the .txt files
        with open(base_dir + doc, "r", encoding="utf-8") as file: # force unicode conversion to keep PCs happy
            text = file.read() # read in the file as a single text string
            all_docs.append(text) # append it to the all_docs list

# lastly, just check the length of all_docs to see if it's 147
len(all_docs)

Next, we'll use the scikit-learn CountVectorizer to count the words in each document:

In [ ]:
# use the scikit-learn countvectorizer to count our words

from sklearn.feature_extraction.text import CountVectorizer

#instantiate CountVectorizer()
cv=CountVectorizer(stop_words='english') # using stopwords this time
 
# this steps returns word counts for the words in your docs 
word_count_vector=cv.fit_transform(all_docs)

# check shape
word_count_vector.shape

Remember: that last line tells us that we have 147 rows, one for each document in the corpus, and 24,930 columns, one for each word (minus single character words, which the tokenizer excludes, as well as the default stopwords, which we've indicated with the stop_words='english' parameter above).

But we also need to keep track of the actual words, not just the counts. The following code will do it:

In [ ]:
# get_feature_names() -- very helpful! 
feature_names = cv.get_feature_names()
 
# shove them all into a dataframe; remember that scikitlearn uses sparse matrices, so we need to make them dense
df = pd.DataFrame(word_count_vector.T.todense(), index=feature_names)

df

In [ ]:
# get a single row as a series for future analysis
# note use .iloc for indices, .loc for label

woman_counts = df.loc["woman"]

In [ ]:
woman_counts.describe()

In [ ]:
woman_counts.plot()

In [ ]:
# sometimes it's just easier to swap the columns and the rows; transpose() does that 

In [ ]:
transposed_df = df.transpose()

In [ ]:
# now we can use even easier indexing for plotting

In [ ]:
transposed_df.plot(y=["woman","man"])

In [ ]:
# look for correlation b/t them
transposed_df.plot(kind="scatter", x="woman", y="man")

In [ ]:
# hm. not much correlation. also, that was slow! 

# can you create a new dataframe with only the records that contain the word "woman"?

In [ ]:
only_woman = # replace w/ your selection code ...



only_woman

In [ ]:
# can you sort the remaining records by how many mentions of "woman" appear?


# move below to LK notebook

sorted_ow = # replace w/ your sorting code



sorted_ow

In [ ]:
# what other questions might you ask? where might you go from here?



## Other resources

* [Greg Reda's Pandas Introduction](http://gregreda.com/2013/10/26/intro-to-pandas-data-structures/) is fantastic and I borrowed many ideas from it. Thanks Greg!
* [A great gist with reminders for Pandas indexing syntax](https://gist.github.com/why-not/4582705)